# Extremely Minimal, Dirty Bleak Scanner implementation 

In [1]:
!pip3 install -r requirements.txt

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


In [2]:
from transitions.extensions.asyncio import AsyncMachine
import asyncio

In [3]:
from bleak import BleakClient, BleakScanner

In [4]:
def detection_callback(device_dict, device, advertisement_data):
    # Here, we're adding device information to the global list
    # Note 'BLEDevice' is a BleakScanner object, which is not JSON serializable, so don't send that to client.
    # But we need it to connect to the device later (simply using the address text doesn't work)
    #ble_devices[device.address] = (device, advertisement_data)
    # we eagerly send the results of the scan to the client through websocket
    # async def eagerly_send_scan_results():
    #     # send `discovered_devices` except for the `BLEDevice` key
    #     print(f"Sending websocket message. Event name: 'scan_reply'. Data: {serialize(ble_devices)}")
    #     await websocket.send(json.dumps({"event": "scan_reply", "data": serialize(ble_devices)}))
    # if websocket is not None:
    #     asyncio.create_task(eagerly_send_scan_results())
    # else:
    #     print("No active websocket connection.")
    # print(f"Advertisement {advertisement_data} found.")
    # print(f"Device {device} found.")
    # print(f"BLE Devices: {device_dict}")
    device_dict[device.address] = (device, advertisement_data)
    return device_dict

In [5]:
from functools import partial
class BleakModel:
    '''
    This class is a transitions.AsyncModel wrapper around the BleakScanner class.
    It is used to scan for Bluetooth Low Energy devices and return the results within the state machine framework.
    '''
    def __init__(self):
        self.ble_devices = {}

    async def bt_scan(self):
        stop_event = asyncio.Event()

        # This will automatically stop the scanner after the given number of seconds
        def stop_scan():
            stop_event.set()
            # async def send_scan_finished_message():
            #     print(f"Sending websocket message. Event name: 'scan_finished'. Data: (empty)")
            #     await websocket.send(json.dumps({"event": "scan_finished", "data": ""}))
            # asyncio.create_task(send_scan_finished_message())

        # Start the scanner with the detection callback
        detection_callback_with_devices = partial(detection_callback, self.ble_devices)
        async with BleakScanner(detection_callback_with_devices) as scanner:
            await asyncio.sleep(3)  # Adjust the sleep time as needed
            stop_scan()

        # Return the list of discovered devices
        return self.ble_devices


In [6]:

model = BleakModel()

transitions = [
    { 'trigger': 'start_scan', 'source': 'Init', 'dest': 'Scanning', 'after': 'bt_scan' }
]
machine = AsyncMachine(model, states=["Init", "Scanning"], transitions=transitions, initial='Init')

In [7]:
# Since we're in a Jupyter notebook, 
# we need to use `create_task()` instead of `get_event_loop().run_until_complete()`
# to use the existing event loop which Jupyter runs on.
asyncio.create_task(model.start_scan())

<Task pending name='Task-5' coro=<AsyncEvent.trigger() running at /Users/tensorturtle/Library/Python/3.9/lib/python/site-packages/transitions/extensions/asyncio.py:166>>

In [8]:
async def print_as_we_go():
    for i in range(20):
        print(f"{len(model.ble_devices)} devices found in {i*0.02} seconds.")
        await asyncio.sleep(0.05)

asyncio.create_task(print_as_we_go())

<Task pending name='Task-7' coro=<print_as_we_go() running at /var/folders/16/dbv855p93hx5m7mgrd8fwscw0000gn/T/ipykernel_10374/1665713095.py:1>>

0 devices found in 0.0 seconds.
1 devices found in 0.02 seconds.
1 devices found in 0.04 seconds.
3 devices found in 0.06 seconds.
3 devices found in 0.08 seconds.
5 devices found in 0.1 seconds.
5 devices found in 0.12 seconds.
6 devices found in 0.14 seconds.
6 devices found in 0.16 seconds.
6 devices found in 0.18 seconds.
6 devices found in 0.2 seconds.
6 devices found in 0.22 seconds.
6 devices found in 0.24 seconds.
6 devices found in 0.26 seconds.
6 devices found in 0.28 seconds.
6 devices found in 0.3 seconds.
6 devices found in 0.32 seconds.
6 devices found in 0.34 seconds.
6 devices found in 0.36 seconds.
6 devices found in 0.38 seconds.
